## Metoda Walidacji Krzyżowej

Jest to jedna z metod estymacji (oszacowania) błędu klasyfikatora. W metodzie tej wykonywany jest podział próby na `v` podzbiorów. `v-1` z nich używamy do uczenia, jeden do
sprawdzenia. Procedurę powtarzamy `v` razy.

Metoda ta jest szczególnie istotna, podczas poszukiwania optymalnych parametrów klasyfikatora.

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from __future__ import print_function
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

Celem zestawu jest znalezienie optymalnego klasyfikator dla danych irys. W tym celu przeszukamy przestrzeń hiperparametrów dla klasyfikatorów kNN i SVM.

### Część 1

Wczytaj dane irys, 
- podziel je na zbiór uczący i testowy w proporcji 7:3 oraz  dokonaj ich standaryzacji metodą `preprocessing.StandardScaler`. Podczas dzielenia ustaw `random_state` na 1234. W metodzie `fit`  obiektu klasy `preprocessing.StandardScaler` uwzględij tylko dane treningowe, natomiast dokonaj standaryzacji zarówno danych testowych, jak i treningowych.

- stwórz zbiór składający się ze wszystkich dostępnych danych i wykonaj dla niego standaryzację. 

In [ ]:
standard_train = None
standard_test = None
train_targets = None
test_targets = None

standard_all = None
all_targets = None

### BEGIN SOLUTION

iris_data = datasets.load_iris()
X = iris_data.data

#Wektor poprawnej klasyfikacji obiektów
y = iris_data.target
y = np.array(y)


#Dzielimy losowo zbiór na dwie części
train, test, train_targets, test_targets = train_test_split(X, y,
                                 test_size=0.30, random_state=1234)



scaler = preprocessing.StandardScaler(with_std = True)
scaler.fit(train)

standard_train = scaler.transform(train)
standard_test = scaler.transform(test)

scaler_all = preprocessing.StandardScaler(with_std = True)
scaler_all.fit(X)

standard_all = scaler_all.transform(X)
all_targets = y
### END SOLUTION


In [ ]:
np.testing.assert_array_almost_equal(np.mean(standard_train, axis=0), [0,0,0,0])
np.testing.assert_array_almost_equal(np.std(standard_train, axis=0), [1,1,1,1])

In [ ]:
np.testing.assert_array_almost_equal(np.mean(standard_all, axis=0), [0,0,0,0])
np.testing.assert_array_almost_equal(np.std(standard_all, axis=0), [1,1,1,1])

### Część 2

Dokonaj klasyfikacji z wykorzystaniem klasyfika kNN dla `k = 1`. Oszacuj błąd klasyfikacji z wykorzystaniem metody walidacji krzyżowej. W tym celu wykorzystaj zbiór `standard_all`. Użyj metody `cross_val_score` z biblioteki `sklearn`. Porownaj wynik walidacji krzyżowej z oszacowaniem dokladności z wykorzystaniem danych testowych. Jakie są czasy obu operacji?

In [ ]:
### BEGIN SOLUTION

clf = KNeighborsClassifier(1,n_jobs=4)
clf.fit(standard_train,train_targets)
%time print("Dokładność na danych testowych",clf.score(standard_test,test_targets ))
%time scores = cross_val_score(clf, standard_all, all_targets, cv=5)
print("Dokładność CV: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print (scores)
### END SOLUTION

### Część 3

Samodzielnie (nie korzystając z funkcji bibliotecznej)wykonaj procedurę walidacji krzyżowej. W pierwszym kroku podziel zbiór danych na 10 części. Wykorzystaj do tego metodę `KFold`. Ustaw `random_state` na 123. 

Następnie metodą split stwórz 10 podziałów na dane testowe i treningowe Dla każdego podziłu (w pętli) trenuj klasyfikator na danych treningowych i oszacuj błęd na danych testowych. Otrzymane wyniki zapisz do tablicy. Po wyjściu z pętli wyznacz średni błąd.

In [ ]:
scores = []

### BEGIN SOLUTION

kf = KFold(n_splits=10, random_state=123)


print(kf)  

for train_index, test_index in kf.split(standard_all):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = standard_all[train_index], standard_all[test_index]
    y_train, y_test = all_targets[train_index], all_targets[test_index]
    clf = KNeighborsClassifier(1,n_jobs=4)
    clf.fit(X_train,y_train)
    scores.append(clf.score(X_test,y_test))
    

### END SOLUTION

print ("Poszczególne wyniki: ", scores)
print ("Błąd wyznaczony procedurą walidacji krzyżowej: ", np.array(scores).mean())

In [ ]:
np.testing.assert_array_almost_equal(scores, [1.0, 1.0, 1.0, 0.9333333333333333, 0.8, 0.8666666666666667, 1.0, 0.8666666666666667, 0.8666666666666667, 1.0])
assert np.array(scores).mean()== 0.9333333333333333

### Część 4

Tym razem w miejsce metody `KFold` użyj `StratifiedKFold`. `random_state=678` Jaki teraz otrzymujesz błąd? Czym różnią się między sobą te dwie metody?

In [ ]:
scores = []

### BEGIN SOLUTION

kf = StratifiedKFold(n_splits=10, random_state=678)
kf.get_n_splits(standard_all)

print(kf)  

for train_index, test_index in kf.split(standard_all, all_targets):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = standard_all[train_index], standard_all[test_index]
    y_train, y_test = all_targets[train_index], all_targets[test_index]
    clf = KNeighborsClassifier(1,n_jobs=4)
    clf.fit(X_train,y_train)
    scores.append(clf.score(X_test,y_test))
    

### END SOLUTION

print ("Poszczególne wyniki: ", scores)
print ("Błąd wyznaczony procedurą walidacji krzyżowej: ", np.array(scores).mean())

In [ ]:
np.testing.assert_array_almost_equal(scores, [1.0, 0.9333333333333333, 1.0, 0.9333333333333333, 0.8666666666666667, 1.0, 0.8, 1.0, 1.0, 1.0])
assert np.array(scores).mean()== 0.9533333333333334

### Część 5

Z wykorzystaniem walidacji krzyżowej znajdź optymalny parametr `k` dla $k \in (1, 20)$. Wykorzystaj metodę przeszukującą podany zakres parametrów `GridSearchCV`.

In [ ]:
clf = KNeighborsClassifier(1,n_jobs=4)
clf.get_params()

In [ ]:
%%time
### BEGIN SOLUTION
k = np.arange(20)+1
tuned_parameters = {'n_neighbors': k}

clf = GridSearchCV(KNeighborsClassifier(n_jobs=4), tuned_parameters, cv=5)
clf.fit(standard_train, train_targets)

### END SOLUTION

In [ ]:
print("Zestaw najlepszych parametrów:")
print()
print(clf.best_params_)
print()
print ("Błąd: ", clf.best_score_)
print()
print("Wyniki")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) dla %r"
            % (mean, std * 2, params))
print()
clf.best_estimator_

In [ ]:
clf.best_estimator_.score(standard_test, test_targets)

### Część 6

Dokonaj klasyfikacji z wykorzystaniem klasyfika SVM dla domyślnych parametrów. Oszacuj błąd klasyfikacji z wykorzystaniem metody walidacji krzyżowej (bez wykorzystywania danych testowych). Użyj metody `cross_val_score` z biblioteki `sklearn`. Porownaj wynik walidacji krzyżowej z oszacowaniem dokladności z wykorzystaniem danych testowych. Jakie są czasy obu operacji?

In [ ]:
### BEGIN SOLUTION

clf = SVC()
clf.fit(standard_train,train_targets)
%time print("Dokładność na danych testowych",clf.score(standard_test,test_targets ))
%time scores = cross_val_score(clf, standard_test, test_targets, cv=10)
print("Dokładność CV: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

### END SOLUTION

### Część 7

Przeskanuj następujący zestaw parametrów dla metody SVM.

In [ ]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [ ]:
%%time
### BEGIN SOLUTION
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
clf = GridSearchCV(SVC(), tuned_parameters, cv=5)
clf.fit(standard_train, train_targets)

### END SOLUTION

In [ ]:
print("Zestaw najlepszych parametrów:")
print()
print(clf.best_params_)

means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
amax = np.argmax(means)
print ("Błąd: ", clf.best_score_)
print()
print("Wyniki")
print()

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) dla %r"
            % (mean, std * 2, params))
print()
